# Задание 9. Контест. Spotify tracks popularity prediction

В этом задании вам предложен набор данных, содержащий информацию о некоторых композициях из сервиса Spotify. Признаковое описание состоит из результатов запросов **GET https:// api.spotify.com/v1/audio-features/{id}**, которые возвращают объекты, которые описывают такие свойства композиций, как танцевальность, акустичность, громкость, темп, а также тональность и некоторые другие. Более подробно про признаки можно прочитать в [описании (англ.)](https://developer.spotify.com/documentation/web-api/reference/#/operations/get-audio-features) такого объекта в официальной документации Spotify. В выборку также добавлены закодированные номера артистов и альбомов (сквозная нумерация).

**ВНИМАНИЕ! Масштаб и абсолютные значения некоторых вещественных признаков были изменены!**

В качестве целевой переменной выбран параметр **popularity** из API Spotify, который принимает целые значения от 0 до 100 и тем выше, чем больше раз трек был прослушан и чем менее давними были эти прослушивания. Более подробно можно прочитать в выдержке из документации Spotify:

> **popularity**

> The popularity of the track. The value will be between 0 and 100, with 100 being the most popular.
The popularity of a track is a value between 0 and 100, with 100 being the most popular. The popularity is calculated by algorithm and is based, in the most part, on the total number of plays the track has had and how recent those plays are.
Generally speaking, songs that are being played a lot now will have a higher popularity than songs that were played a lot in the past. Duplicate tracks (e.g. the same track from a single and an album) are rated independently. Artist and album popularity is derived mathematically from track popularity. Note that the popularity value may lag actual popularity by a few days: the value is not updated in real time.

**ВНИМАНИЕ! Масштаб и абсолютные значения популярности были изменены!**

В качестве метрики качества используется **rMSE (root mean squared error)** -- корень из среднеквадратической ошибки (**обратите внимание на параметр squared=False в вызовах mean_squared_error**).

Вам необходимо разработать модель машинного обучения для восстановления зависимости **популярности** композиции от ее вышеописанных свойств.

В папке с заданием помимо этого файла и примера корректной посылки находятся 2 файла с данными -- **train.csv** и **test.csv**, которые содержат, соответственно, тренировочную выборку (с известными значениями целевой переменной в поле **popularity**) и признаковую часть тестовой выборки.


В качестве ответа на это задание вы должны предоставить **Kaggle-ноутбук** (как создать такой ноутбук, читайте ниже), который:
1. генерирует на выходе **csv-файл** со столбцом предсказанных популярностей для композиций из тестовой выборки и отправляет его в систему Kaggle. Пример такого файла находится в папке с заданием (**sample_submission.csv**);
2. разрешает чтение пользователю [Sergey Serov](https://www.kaggle.com/ssserov/account).

**НЕВЫПОЛНЕНИЕ ЛЮБОГО ИЗ УКАЗАННЫХ ПУНКТОВ ПРИВЕДЁТ К ОЦЕНИВАНИЮ ЗАДАНИЯ В 0 БАЛЛОВ!**

**Как создать и отправить корректный Kaggle-ноутбук:**

1. На странице соревнования перейдите на вкладку **Code** и нажмите **New Notebook**.

2. **Никакие дополнительные данные для выполнения задания загружать не нужно** (но это не запрещено). Путь, по которому автоматически находятся необходимые файлы с данными, можно посмотреть, выполнив первую ячейку и изучив ее вывод.

3. Дать права на чтение ноутбука пользователю [Sergey Serov](https://www.kaggle.com/ssserov/account). Для этого в верхней панели ноутбука нужно нажать кнопку **Share**, далее выбрать **Add collaborators** и в поиске найти пользователя Sergey Serov (вместо owner будет написано collaborator). Не забудьте сохранить изменения кнопкой **Save**.



После правильного действия Вы увидите:


4. Для того, чтобы предсказания, полученные kaggle-ноутбуком были корректно учтены системой, он должен сохранять их следующей командой **submission.to_csv("/kaggle/working/submission.csv", index_label="index")**, где **submission** -- ваш датафрейм с предсказаниями (как в примере ниже).

5. Для отправки ноутбука в правой его панели выберите вкладку **Competitions**, нажмите на кнопку **Submit**, по желанию введите название и описание посылки и подтвердите нажатием кнопки **Submit**.


**Далее в этом ноутбуке покажем пример формирования csv-файла с предсказаниями популярности для композиций из тестовой выборки.**

Для начала импортируем библиотеки и загрузим данные из файлов **train.csv** и **test.csv**.

In [31]:
import pandas as pd
from sklearn.metrics import mean_squared_error

In [32]:
# Используйте эти пути для запуска ноутбука на Kaggle
PATH_TO_KAGGLE_TRAIN = "/kaggle/input/cmc-ml-spotify-tracks-popularity-prediction/train.csv"
PATH_TO_KAGGLE_TEST = "/kaggle/input/cmc-ml-spotify-tracks-popularity-prediction/test.csv"
PATH_TO_KAGGLE_SUBMISSION = "/kaggle/working/submission.csv"

# Если запускаете на своем компьютере, то впишите актуальные пути
PATH_TO_TRAIN = "data/train.csv"
PATH_TO_TEST = "data/test.csv"
PATH_TO_SUBMISSION = "data/submission.csv"

#train = pd.read_csv(PATH_TO_KAGGLE_TRAIN)
#test = pd.read_csv(PATH_TO_KAGGLE_TEST)

train = pd.read_csv(PATH_TO_TRAIN) #Delete me
test = pd.read_csv(PATH_TO_TEST)


X_train = train.drop(["index", "popularity"], axis=1)
y_train = train["popularity"]

train.head()

,index,composer,album,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,popularity
0,0,35,5187,6538.248,0.333312,2,-17.541540,0,0.034528,-0.432689,0.002920,567.788,0.54487,92.65497,347863.20,4,35
1,1,206,799,6323.376,0.397296,0,-16.723045,1,0.030451,-0.357520,0.000000,2196.688,0.72203,104.02050,205176.03,4,5
2,2,318,1152,7305.648,0.511872,2,-21.101915,1,0.032365,-0.423232,0.000030,474.708,0.74366,101.36628,610613.52,4,8
3,3,325,685,7090.776,0.583296,2,-15.194040,1,0.031200,-0.383050,0.000002,1438.086,0.66435,100.36932,282403.17,4,5
4,4,230,606,6159.664,0.685224,9,-16.344315,1,0.420160,-0.334210,0.000000,4346.836,0.80752,72.41445,76964.37,3,16


Затем создадим модель и обучим ее на тренировочной выборке.

In [33]:
from lightgbm import LGBMRegressor
model = LGBMRegressor(n_estimators=5000, max_depth=5, learning_rate=0.1)
model.fit(X_train, y_train)

LGBMRegressor(max_depth=5, n_estimators=5000)

Вычислим ошибку модели на тренировочной выборке.

In [34]:
print(f"Train rMSE: {mean_squared_error(y_train, model.predict(X_train), squared=False)}")

Train rMSE: 5.501528436309721


В заключение получим столбец предсказаний популярности для тестовой выборки и сохраним его в виде csv-файла (обратите внимание, что в выходном файле должно быть два столбца -- **index** и **popularity**).

In [37]:
submission = pd.DataFrame({"index": test["index"], 
                           "popularity": model.predict(test.drop("index", axis=1))})

In [36]:
submission.to_csv(PATH_TO_SUBMISSION, index=False)